In [17]:
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp,sqrt;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [18]:
V_max = 4.2;        V_min = 2.5;
Exp_1  = pybamm.Experiment(
    [ (
        f"Discharge at 1 C until {V_min} V", 
        f"Charge at 1 C until {V_max} V", 
        f"Hold at {V_max} V until C/5")    ] * 1 )  
Model_0  = pybamm.lithium_ion.DFN(options={
    "solvent diffusion": "double spatial consume w refill",
    "electrolyte conductivity": "full"  ,
    }) 
Model_1  = pybamm.lithium_ion.DFN(options={
    "solvent diffusion": "double spatial consume w refill",
    "electrolyte conductivity": "sol full"  ,
    }) 

In [19]:
def EC_diffusivity_5E_5(c_e, c_EC , T):
    D_ec_dim = (
        (c_EC >= 0 ) * 5e-5 
        +  (c_EC < 0 ) * 0 
    )
    return D_ec_dim
def electrolyte_conductivity_Andrew2022(x,y, T):# x:Li+,y:ec
    p00 =     -0.2524;
    p10 =    0.001402;
    p01 =   0.0001142 ;
    p20 =  -5.379e-07  ;
    p11 =  -1.399e-08 ;
    p02 =  -8.137e-09  ;
    kai  = (
        (p00 + p10*x + p01*y + p20*x*x + p11*x*y + p02*y*y)
    )
    return kai
def electrolyte_conductivity_Nyman2008Exp(c_e,c_EC, T):
    sigma_e = (
        0.1 * 0.06248 * (1+298.15-0.05559) * 
        (c_e/1e3) * (1 - 3.084 *sqrt(c_e/1e3) 
        + 1.33 *(1+ 0.03633 *(exp(1000/298.15))*c_e/1e3)   ) 
        / (1+(c_e/1e3)**4*( 0.00795 *exp(1000/298.15))) 
    )
    return sigma_e
def electrolyte_diffusivity_Nyman2008Exp(c_e,c_EC, T):
    D_c_e = (
        6 * exp( -1 *(c_e/1000)) 
        * exp(-5/298.15) 
        * exp(-95/298.15*(c_e/1000)) * 1e-10 
    )
    return D_c_e
def electrolyte_conductivity_Nyman2008Constant(c_e,c_EC, T):
    
    sigma_e = (c_e <= 2000) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)
    ) + (c_e > 2000) *  (
        0.1297 * (2000 / 1000) ** 3 - 2.51 * (2000 / 1000) ** 1.5 + 3.329 * (2000 / 1000)
    )
    return sigma_e
def electrolyte_diffusivity_Nyman2008Constant(c_e,c_EC, T):
    D_c_e = (
        (c_e <= 2000) * (
            8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10) 
        + (c_e > 2000) * (
        8.794e-11 * (2000 / 1000) ** 2 - 3.972e-10 * (2000 / 1000) + 4.862e-10)
    )
    return D_c_e

In [20]:
Para_0=pybamm.ParameterValues("Li2023_ECdrag")
Para_0.update({'Negative electrode porosity':0.15})
Para_0.update({"EC diffusivity in electrolyte [m2.s-1]":EC_diffusivity_5E_5})
Para_0.update({"Electrolyte conductivity [S.m-1]":electrolyte_conductivity_Nyman2008Constant})
Para_0.update({"Electrolyte diffusivity [m2.s-1]":electrolyte_diffusivity_Nyman2008Constant})
Para_0.update({"EC Lithium ion cross diffusivity [m2.s-1]":1.5e-12})

c_e = Model_0.variables["Electrolyte concentration [mol.m-3]"]
c_EC= Model_0.variables["EC concentration [mol.m-3]"]
T = Model_0.variables["Cell temperature [K]"]
D_e = Para_0["Electrolyte diffusivity [m2.s-1]"]
sigma_e = Para_0["Electrolyte conductivity [S.m-1]"]
Xi = Para_0["EC transference number"]
Model_0.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
Model_0.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
Model_0.variables["EC transference number"] = Xi(c_e,c_EC, T)

Para_1=pybamm.ParameterValues("Li2023_ECdrag")
Para_1.update({'Negative electrode porosity':0.15})
# Para_1.update({"EC diffusivity in electrolyte [m2.s-1]":EC_diffusivity_5E_10})
Para_1.update({"Electrolyte conductivity [S.m-1]":electrolyte_conductivity_Nyman2008Constant})
Para_1.update({"Electrolyte diffusivity [m2.s-1]":electrolyte_conductivity_Nyman2008Constant})
Para_1.update({"EC Lithium ion cross diffusivity [m2.s-1]":1.5e-12})
c_e = Model_1.variables["Electrolyte concentration [mol.m-3]"]
c_EC= Model_1.variables["EC concentration [mol.m-3]"]
T = Model_1.variables["Cell temperature [K]"]
D_e = Para_1["Electrolyte diffusivity [m2.s-1]"]
sigma_e = Para_1["Electrolyte conductivity [S.m-1]"]
Xi = Para_1["EC transference number"]
Model_1.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
Model_1.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
Model_1.variables["EC transference number"] = Xi(c_e,c_EC, T)

In [21]:
Sim_0    = pybamm.Simulation(
    Model_0, experiment = Exp_1,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)       
Sol_0    = Sim_0.solve()

In [22]:
Sim_1    = pybamm.Simulation(
    Model_1, experiment = Exp_1,
    parameter_values = Para_1,
    solver = pybamm.CasadiSolver(),)       
Sol_1    = Sim_1.solve()

In [23]:
label = ["Normal DFN","Double transport + Sol affect potential",] 
output_variables3 = [
    "Terminal voltage [V]", 
    "Electrolyte potential [V]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "EC transference number",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]"
]
quick_plot = pybamm.QuickPlot(
    [Sol_0,Sol_1], output_variables3,label,  #variable_limits='fixed',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=2.2834031423898358, step=0.02283403142389836…